Notebook by Ayushman Buragohain



## seresnext50_32x4d + Label-Smoothing + Ranger + Flat-Anneal

In [ ]:
import os
import wandb
import albumentations as A
import timm

from fastai.vision.all import *
from fastai.callback.wandb import *

from cassava_utils import *

In [ ]:
SEED        = 42
IDX         = generate_random_id()
OOF_FOLD    = 0
NUM_CLASSES = 5
CSV_PATH    = '../data/fold_df.csv'
IMAGE_DIR   = '../../train_images/'
INPUT_DIMS  = 224
MODEL       = 'seresnext50_32x4d'
PROJECT     = 'kaggle-leaf-disease-fastai-runs'
SAVE_NAME   = f"{MODEL}-fold={OOF_FOLD}-{IDX}"

set_seed(SEED, reproducible=True)

Ts = [MODEL, 'ranger', 'label-smoothing', str(INPUT_DIMS) + 'mixup' + 'mish']
Ns = f'{MODEL} + Ranger + LabelSmoothing + MixUp + Mish'
run = wandb.init(project=PROJECT, tags=Ts, notes=Ns,)

In [ ]:
data = get_dataset(CSV_PATH, IMAGE_DIR, OOF_FOLD, shuffle=True)
data.head()

In [ ]:
TRAIN_AUGS = A.Compose([
    A.OneOf([A.RandomResizedCrop(INPUT_DIMS, INPUT_DIMS), A.CenterCrop(INPUT_DIMS, INPUT_DIMS)], p=0.5),
    A.Resize(INPUT_DIMS, INPUT_DIMS, p=1.0),
    A.HorizontalFlip(),
    A.OneOf([A.Rotate(), A.VerticalFlip()], p=0.5),
    A.OneOf([A.Flip(), A.IAAPerspective(), A.ShiftScaleRotate()], p=0.5),
    A.RandomBrightnessContrast(0.1, 0.1, p=0.5),
    A.OneOf([A.CLAHE(), A.HueSaturationValue(0.2, 0.2, 0.2), A.IAASharpen()], p=0.5),
    A.CoarseDropout(p=0.5),
    A.Cutout(p=0.5, num_holes=15),
])

VALID_AUGS = A.Compose([A.Resize(INPUT_DIMS, INPUT_DIMS, p=1.0)])

ITEM_TFMS  = AlbumentationsTransform(TRAIN_AUGS, VALID_AUGS)
BATCH_TFMS = [Normalize.from_stats(*imagenet_stats)]

cassava = DataBlock(blocks=(ImageBlock, CategoryBlock),
                  splitter=ColSplitter(),
                  get_x=lambda o: o['filePath'], 
                  get_y=lambda o: o['label'], 
                  item_tfms=ITEM_TFMS, 
                  batch_tfms=BATCH_TFMS)

dls = cassava.dataloaders(data, bs=32)

In [ ]:
dls.train.show_batch(max_n=3,figsize=(12,6))

In [ ]:
dls.valid.show_batch(max_n=3,figsize=(12,6))

In [ ]:
@delegates(Learner)
def timm_learner(dls, m_name:str, cut:int, c:int, state:str=None, init:bool=True, 
                pretrained:bool=True, act:callable=nn.ReLU(inplace=True), modifier=None, **kwargs):
    'create a fast.ai learner instance from timm'
    encoder = timm.create_model(m_name, pretrained=pretrained, )
    
    model  = TransferLearningModel(encoder, num_classes=c, cut=cut, act=act, init=init)
    
    if modifier is not None: 
        for mds in modifier :  mds(model)
            
    if state is not None: model.load_state_dict(torch.load(state)) 
    
    learn = Learner(dls, model, **kwargs)
    return learn

In [ ]:
cb_list = [WandbCallback(seed=SEED, log_preds=False, log_model=False), TerminateOnNaNCallback()]

learn = timm_learner(dls, MODEL, cut=-2, c=dls.c, pretrained=True, init=True,
                    metrics=[accuracy], splitter=custom_splitter,
                    opt_func=ranger, loss_func=LabelSmoothingCrossEntropy(),
                    cbs=cb_list).to_fp16()

In [ ]:
learn.model

In [ ]:
learn.freeze()
learn.lr_find()

In [ ]:
learn.freeze()
learn.fit_flat_cos(2, slice(2e-05), wd=0.1, cbs=[MixUp(), GradientAccumulation(n_acc=4*dls.bs)])

In [ ]:
learn.save('stage-1')

In [ ]:
learn = learn.load('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
cb_list = [MixUp(), GradientAccumulation(n_acc=4*dls.bs)]
learn.fit_flat_cos(15, slice(6e-04), wd=0.1, cbs=cb_list, pct_start=0.72)

In [ ]:
cb_list = [MixUp(), GradientAccumulation(n_acc=4*dls.bs), SaveModelCallback(monitor='accuracy')]
learn.fit_flat_cos(15, slice(6e-04), wd=0.1, cbs=cb_list, pct_start=0.72)